3. Implemente el algoritmo de Metrópolis Hastings para hacer el ajuste de un histograma cuyas frecuencias relativas siguen una distribución normal.

In [10]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [11]:
file = 'Gaussiana.csv'
url = 'https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv'

In [12]:
data = pd.read_csv(url,sep=",",engine='python')
X = data.x

In [13]:
transition_model = lambda x: [np.random.normal(x[0],0.5,(1,)),np.random.normal(x[1],0.5,(1,))]

In [14]:
def prior(x):
    
    if(3.<=x[0] <=5.)and(0.5<=x[1] <=3.5):
        return 1
    else:
        return 0

In [15]:
def Likelihood(x,data):

  return (np.exp(-0.5*(x[0]-data)**2/x[1]**2))/(np.sqrt(2*np.pi*x[1]**2))

In [16]:
def LogLikelihood(x,data):
    return np.sum(np.log(Likelihood(x,data)))

In [17]:
def acceptance(x, x_new):
    if x_new>x:
        return True
    else:
        accept=np.random.uniform(0,1)

        return (accept < (np.exp(x_new-x)))

In [20]:
def metropolis_hastings(likelihood_computer,prior, transition_model, param_init,iterations,data,acceptance):
   
    x = param_init
    accepted =np.zeros((iterations,2))
   
    for i in  tqdm(range(iterations)):
        x_new =  transition_model(x)    
        x_lik = likelihood_computer(x,data)
        x_new_lik = likelihood_computer(x_new,data) 
        if (acceptance(x_lik + np.log(prior(x)),x_new_lik+np.log(prior(x_new)))):            
            x = x_new
            accepted[i]=x
        else:
            accepted[i]=x    
                
    return accepted[1000:,:]

In [21]:
accepted = metropolis_hastings(LogLikelihood,prior,transition_model,[2,3], 20000,X,acceptance)

  0%|          | 0/20000 [00:00<?, ?it/s]C:\Users\USUARIO\AppData\Local\Temp\ipykernel_30364\2092399341.py:10: RuntimeWarning: divide by zero encountered in log
  if (acceptance(x_lik + np.log(prior(x)),x_new_lik+np.log(prior(x_new)))):
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_30364\3648019792.py:7: RuntimeWarning: invalid value encountered in scalar subtract
  return (accept < (np.exp(x_new-x)))
  0%|          | 74/20000 [00:00<01:26, 231.48it/s]


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.

In [ ]:
Best_params=np.array([np.percentile(accepted[:,0],50),np.percentile(accepted[:,1],50)])

In [ ]:
_x = np.linspace(np.min(X),np.max(X),100)

plt.hist(X,density=True)
plt.plot(_x,Likelihood(Best_params,_x))

In [22]:
Best_params

NameError: name 'Best_params' is not defined

In [ ]:
Percentile84=np.array([np.percentile(accepted[:,0],84),np.percentile(accepted[:,1],84)])
Percentile16=np.array([np.percentile(accepted[:,0],16),np.percentile(accepted[:,1],16)])

Mu_error=[Percentile84[0]-Best_params[0],Percentile16[0]-Best_params[0]]
Sigma_error=[Percentile84[1]-Best_params[1],Percentile16[1]-Best_params[1]]

Mu_error,Sigma_error

$$
\hat{\mu}=4.01_{0.18}^{0.18} at68\%CL
$$

$$
\hat{\sigma}=1.83_{0.11}^{0.13} at68\%CL
$$